# Time-dependent PDEs

So far we've seen ODEs, and looked at time-integration techniques, and then PDEs but we mostly focused on stationary problems. Now we will combine the two and look at time-dependent PDEs. As a model problem we will consider the [*heat equation*](https://en.wikipedia.org/wiki/Heat_equation) which models the diffusion of heat in a material with some given thermal conductivity

$$
\partial_t u - \alpha \nabla^2 u = 0
$$

augmented with appropriate initial and boundary conditions. We will look at both implicit and explicit time integration schemes for this equation, starting with explicit schemes.

In [ ]:
%matplotlib notebook
from matplotlib import pyplot
import numpy
pyplot.style.use('ggplot')
from collections import namedtuple
Point = namedtuple("Point", ("x", "y"))

class Grid(object):
    def __init__(self, Nx, Ny, P0=Point(0,0), P1=Point(1,1)):
        X0, Y0 = P0
        X1, Y1 = P1
        self.W = X1 - X0
        self.H = Y1 - Y0
        self.Nx = Nx
        self.Ny = Ny
        x = numpy.linspace(X0, X1, self.Nx)
        y = numpy.linspace(Y0, Y1, self.Ny)
        self.XY = numpy.meshgrid(x, y, indexing="ij")
    
    @property
    def ndof(self):
        return self.Nx*self.Ny

    @property
    def hx(self):
        return self.W/(self.Nx - 1)
    
    @property
    def hy(self):
        return self.H/(self.Ny - 1)

    def alpha(self, i, j):
        return i*self.Ny + j

    def new_vector(self, components=1):
        vec = numpy.zeros(self.Nx*self.Ny*components, dtype=float)
        shape = (self.Nx, self.Ny)
        if components > 1:
            shape = shape + (components, )
        return vec.reshape(shape)
    
    def contourf(self, u, levels=11, ax=None):
        U = u.reshape(self.Nx, self.Ny)
        if ax is None:
            pyplot.figure()
            contours = pyplot.contourf(*self.XY, U, levels)
            pyplot.colorbar(contours)
        else:
            contours = ax.contourf(*self.XY, U, levels)
            pyplot.colorbar(contours)
        return contours
        
    def quiver(self, u, colour=None, ax=None):
        U = u.reshape(self.Nx, self.Ny, 2)
        if ax is None:
            pyplot.figure()
            quiver = pyplot.quiver
        else:
            quiver = ax.quiver
        if colour is None:
            vecs = quiver(*self.XY, U[..., 0], U[..., 1])
        else:
            vecs = quiver(*self.XY, U[..., 0], U[..., 1], colour)
            pyplot.colorbar(vecs)
        return vecs

## An explicit scheme

We will first discretise the time derivative. Recall the general form of an ODE is

$$
\partial_t u = f(t, u)
$$

where here we have

$$
f(t, u) = \alpha \nabla^2 u.
$$

In an explicit scheme, we evaluate $f(u)$ at the beginning of the timestep. We'll start with explicit Euler

$$
u^{n+1} = u^n + \Delta t \alpha \nabla^2 u^n.
$$

So given an initial condition $u^0$ we just need to be able to compute $\alpha \nabla^2 u^0$ and add it on to get the value at the next timestep.

Let's solve this problem on the square domain $\Omega = [0, 1] \times [0, 1]$ with the boundary conditions

$$
\begin{aligned}
u &= 1 && x = 0, y \in [0.25, 0.75]\\
u &= 0 && x = 1, y \in [0.6, 0.8]\\
\nabla u \cdot n &= 0 && \text{otherwise}.
\end{aligned}
$$

We can think of this as modelling a 2D room with a radiator on one wall, a window on the other, and perfectly insulating (ha!) walls everywhere else.

In [ ]:
import numba

@numba.jit(nopython=True)
def f(un, f_, Nx, Ny, hx, hy, stencil):
    for i in range(Nx):
        for j in range(Ny):
            f_[i, j] = 0
            # Dirichlet boundary
            if i == 0 and 0.25 <= j*hy <= 0.75:
                f_[i, j] = 0
            elif i == Nx - 1 and 0.6 <= j*hy <= 0.8:
                f_[i, j] = 0
            else:
                for idx, (i_, j_) in enumerate([(i-1, j), (i, j-1), (i, j), (i, j+1), (i+1, j)]):
                    # Homogeneous Neumann everywhere else: i-1 -> i+1 (i = 0), i+1 -> i-1 (i = Nx - 1), etc...
                    i_ = (Nx - 1) - abs(Nx - 1 - abs(i_))
                    j_ = (Ny - 1) - abs(Ny - 1 - abs(j_))
                    f_[i, j] += stencil[idx] * un[i_, j_]
    return f_

Notice how on the Dirichlet boundary, we set the update function to return zero. This way, as long as our initial condition satisfies the boundary conditions, it will do so for all time. For the homogeneous Neumann condition, we implement the symmetric "reflected" condition (rather than a one-sided difference).

Let's go ahead and integrate this.

In [ ]:
def setup(N):
    grid = Grid(N, N)
    u = grid.new_vector()
    # Initial condition, 1 on the right boundary when y \in [0.25, 0.75]
    for j in range(grid.Ny):
        if 0.25 <= j*grid.hy <= 0.75:
            u[0, j] = 1      
    return grid, u

In [ ]:
def explicit_euler(u0, dt, grid, alpha=1, T=5):
    us = [u0]
    ts = [0]
    update = numpy.zeros_like(u0)
    u = u0
    t = 0
    # Notice how the sign is flipped relative to -\nabla^2 (since we have \partial_t u = +\nabla^2 u)
    stencilx = 1/grid.hx**2 * numpy.array([1, 0, -2, 0, 1])
    stencily = 1/grid.hy**2 * numpy.array([0, 1, -2, 1, 0])
    stencil = stencilx + stencily    
    while t < T:
        update = f(u, update, grid.Nx, grid.Ny, grid.hx, grid.hy, stencil)
        if numpy.linalg.norm(update, numpy.inf) < 1e-10:
            # Terminate if we've reached a steady-state
            break
        # Explicit Euler: u <- u + dt f(u)
        u = u + dt*alpha*update
        us.append(u)
        t += dt
        ts.append(t)
    return ts, us

Now we're ready to integrate the equation, let's try on a relatively coarse grid.;

In [ ]:
N = 11
alpha = 1
grid, u = setup(N)
dt = 0.00252
ts, us = explicit_euler(u, dt, grid, alpha=alpha, T=10)

In [ ]:
grid.contourf(us[-1], levels=20);

This looks like the solution I'm expecting, but the timestep is *very* small. I only have 10 cells in each direction.

Let's see what happens when we make the timestep bigger.

In [ ]:
N = 11
alpha = 1
grid, u = setup(N)
dt = 0.00255
ts, us = explicit_euler(u, dt, grid, alpha=alpha, T=10)

In [ ]:
grid.contourf(us[-1], levels=40);

## Instability for large timesteps

Uh-oh, this looks bad. What's going on? We have hit the [CFL](https://en.wikipedia.org/wiki/Courant–Friedrichs–Lewy_condition) constraint for this PDE.

This property of a timestepping scheme, named after three mathematicians, Courant, Friedrichs, and Lewy, provides us with a rule for determining an appropriate maximum timestep given a particular spatial discretisation. An intuition for what is going on is that the *physical* equation has some domain of dependence. A region of the solution at time $t$ affects some other region of the solution at $t + \Delta t$. If our numerical scheme fails to capture this dependence, we get bad behaviour.

In other words, if we pick a timestep that is too large, information can propagate "too fast" in our numerical simulation.

The CFL condition was developed in the analysis of advection equations

$$
\partial_t u - w \cdot \nabla u = 0.
$$

For which we have the constraint (with $w = 1$)

$$
\frac{\Delta t}{\Delta x} \le 1.
$$

That is, I can't push information more than a single cell in one timestep.

For the heat equation, the restriction is much tighter, we need

$$
\frac{\Delta t}{(\Delta x)^2} \le c
$$

with $c$ some (dimension-dependent) constant. In two dimensions, for explicit Euler, we have $c = 0.25$.

### Eigenvalue analysis

How did I arrive at this magic constant? Recall that the *stability region* for explicit Euler is the unit circle centred at -1 in the complex plane. A necessary condition for stability of the timestepping scheme applied to the scalar Dahlquist test equation

$$
\partial_t u = \lambda u
$$

which, discretised with explicit Euler gives

$$
u^{n+1} = u^n + \lambda\Delta t u^n,
$$

is that $-2 \le \lambda \Delta t < 0$.

How can we apply this same idea here, when we have

$$
\partial_t u = \nabla^2 u
$$

or, discretised

$$
u^{n+1} = u^n + \Delta t \nabla^2 u^n?
$$

For this operator, we can find the bound by considering the *eigenvalues* of $\nabla^2$. If we can find them, we can replace the discretised operator by a diagonal one (with the eigenvalues on the diagonal), and then treat each equation separately.

Let's go ahead and discretise $\nabla^2$ and look at the eigenvalues.

In [ ]:
def laplacian(grid):
    ndof = grid.ndof
    A = numpy.zeros((ndof, ndof))
    X, Y = grid.XY
    Nx = grid.Nx
    Ny = grid.Ny
    stencilx = 1/grid.hx**2 * numpy.array([1, 0, -2, 0, 1])
    stencily = 1/grid.hy**2 * numpy.array([0, 1, -2, 1, 0])
    stencil = stencilx + stencily
    for i in range(grid.Nx):
        for j in range(grid.Ny):
            row = grid.alpha(i, j)
            # Dirichlet boundary
            if i == 0 and 0.25 <= j*grid.hy <= 0.75:
                A[row, row] = 0
            elif i == grid.Nx - 1 and 0.6 <= j*grid.hy <= 0.8:
                A[row, row] = 0
            else:
                indices = [(i-1, j), (i, j-1), (i, j), (i, j+1), (i+1, j)]
                i_ = lambda i_: (Nx - 1) - abs(Nx - 1 - abs(i_))
                j_ = lambda j_: (Ny - 1) - abs(Ny - 1 - abs(j_))
                cols = [grid.alpha(i_(i), j_(j)) for i, j in indices]
                for c, s in zip(cols, stencil):
                    A[row, c] += s
    return A

In [ ]:
grid = Grid(11, 11)
A = laplacian(grid)

We're interested in the *smallest* (most negative) eigenvalue

In [ ]:
evals = numpy.linalg.eigvals(A)
evals.min()

We need, when multiplying this by $\Delta t$ to arrive at a number larger than -2. Which implies

In [ ]:
dt = -2/evals.min()
dt

So $\Delta t = 0.0025$ is right on the edge of stability for our method (hence the problem blowing up with $\Delta t = 0.0026$).

What is the relationship we need between $\Delta x$ and $\Delta t$? The most negative eigenvalue scales with $\frac{1}{(\Delta x)^2}$, and so we need

$$
\frac{\Delta t}{(\Delta x)^2} = \text{const}.
$$

Each time we double the spatial resolution we must reduce the timestep by a factor of four!

### Bounding the eigenvalues of a regular stencil

For the stencils we see in the course, we can put a bound on the eigenvalues (and in particular the smallest one) using a remarkable theorem due to [Gershgorin](https://en.wikipedia.org/wiki/Gershgorin_circle_theorem).

For *any* square matrix $A$ with entries $a_{ij}$, write

$$
R_i = \sum_{j \ne i} |a_{ij}|
$$

(the sum of the absolute value of the off-diagonal entries), and define the disc

$$
D(a_{ii}, R_i) = \{z \in \mathbb{C} : |z - a_{ii}| \le R_i\}
$$

(that is, a circle centered at $a_{ii}$ with radius $R_i$).

Then every eigenvalue of $A$ is contained in at least one of these discs.

In [ ]:
from matplotlib.patches import Circle
from matplotlib.collections import PatchCollection

def gershgorin(A):
    n = len(A)
    evals = numpy.linalg.eigvals(A)
    patches = []
    # draw discs
    seen = set()
    for i in range(n):
        xi = numpy.real(A[i,i])
        yi = numpy.imag(A[i,i])
        ri = numpy.sum(numpy.abs(A[i,:])) - numpy.abs(A[i,i]) 
        if (xi, yi, ri) in seen:
            continue
        circle = Circle((xi, yi), ri)
        patches.append(circle)
        seen.add((xi, yi, ri))
    fig, ax = pyplot.subplots()
    p = PatchCollection(patches, alpha=0.1)
    ax.add_collection(p)
    pyplot.plot(numpy.real(evals), numpy.imag(evals),' o')
    pyplot.axis('equal')
    return fig

In [ ]:
gershgorin(A);

We can see that this isn't a very good estimate of many of the eigenvalues, but it's quite good for the minimal one.

So, if I give you a stencil

$$
\frac{1}{h_x^2}\begin{bmatrix}-1 & 2 & -1\end{bmatrix}
$$

we can immediately say that the maximal eigenvalue will be less than or equal to $\frac{4}{h_x^2}$, and the minimal one will be greater than or equal to zero.

In [ ]:
example = numpy.asarray([[5., 3., 2.], 
                         [4., 6., 5.],
                         [-3., 1., 4.]])
gershgorin(example);

## Breaking through the timestep restriction

Our only chance of being able to take larger timesteps is to increase the size of the stability region. We can try and do so with explicit methods, but we will *always* run into the timestep constraint eventually (since no explicit method contains an unbounded stability region.

Instead, we turn to *implicit* methods. We're now going to have to invert the operator at every timestep, hence our interest in different methods for doing so. We'll do implicit Euler first, for which the discretised problem looks like

$$
\mathbb{I} u^{n+1} - \Delta t \nabla^2 u^{n+1} = u^n.
$$

Rearranging, we obtain

$$
u^{n+1} = (\mathbb{I} - \Delta t \nabla^2)^{-1} u^n
$$

so our update step is to invert an operator onto the old state, rather than applying the operator to the state.

In [ ]:
import scipy.linalg

def implicit_euler(u0, dt, grid, alpha=1, T=5):
    A = dt*alpha*laplacian(grid)
    I = numpy.eye(len(A))
    op = I - A
    lu, piv = scipy.linalg.lu_factor(op)
    t = 0
    us = [u0]
    ts = [t]
    u = u0
    while t < T:
        u = scipy.linalg.lu_solve((lu, piv), u.flat).reshape(u.shape)
        t += dt
        us.append(u)
        ts.append(t)
        if numpy.linalg.norm(us[-2] - us[-1], numpy.inf) < 1e-10:
            break
    return ts, us

In [ ]:
N = 21
grid, u = setup(N)
dt = 1
ts, us = implicit_euler(u, dt, grid, T=100)

In [ ]:
grid.contourf(us[-1], levels=20);